<a href="https://colab.research.google.com/github/garfield-gray/Optimization/blob/main/Convex/IntProg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IntProg

In [22]:
from scipy.optimize import LinearConstraint, milp
import numpy as np

In [23]:
c = np.array([3, 4])
A = np.array([[-3, -1], [-1, -2]])
b_u = np.array([-4, -4])
b_l = np.full_like(b_u, -np.inf, dtype=float)
# b_l = np.full_like(b_u, 0, dtype=float)

In [24]:
constraints = LinearConstraint(A, b_l, b_u)

In [25]:
integrality = np.ones_like(c)
res = milp(c=c, constraints=constraints, integrality=integrality)
res.x

array([2., 1.])

In [26]:
res = milp(c=c, constraints=constraints)  # OR:
# from scipy.optimize import linprog; res = linprog(c, A, b_u)
res.x

array([0.8, 1.6])

# Linprog

In [27]:
from scipy.optimize import linprog
c = [3, 4]
A = [[-3, -1], [-1, -2]]
b = [-4, -4]
x0_bounds = (0, None)
x1_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds])
res.fun


8.8

In [28]:
res.x

array([0.8, 1.6])

In [29]:
res.message

'Optimization terminated successfully. (HiGHS Status 7: Optimal)'

# The Algorithm

In [30]:
import numpy as np

def simplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)

    # Create the initial tableau
    tableau = np.zeros((m + 1, n + m + 1))

    # Fill the tableau with coefficients
    tableau[:-1, :-1] = np.hstack((A, np.eye(m)))
    tableau[:-1, -1] = b
    tableau[-1, :n] = -c

    # The main simplex algorithm
    while True:
        # Check if we have an optimal solution (all entries in the objective row are non-negative)
        if np.all(tableau[-1, :-1] >= 0):
            break

        # Pivot column (most negative entry in the objective row)
        pivot_col = np.argmin(tableau[-1, :-1])

        # Pivot row
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, pivot_col])
        valid_ratios = ratios[tableau[:-1, pivot_col] > 0]
        pivot_row = np.where(ratios == valid_ratios.min())[0][0]

        # Pivot element
        pivot_element = tableau[pivot_row, pivot_col]

        # Update the pivot row
        tableau[pivot_row] /= pivot_element

        # Update the other rows
        for i in range(m + 1):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    # Extract solution
    solution = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col == 0) == m - 1:
            solution[i] = tableau[np.where(col == 1)[0][0], -1]

    # Optimal value
    optimal_value = -tableau[-1, -1]

    return solution, optimal_value

# # Example Usage
# c = np.array([3, 4])
# A = np.array([[-3, -1], [-1, -2]])
# b = np.array([-4, -4])

c = np.array([3, 2])
A = np.array([[1, 2], [1, -1]])
b = np.array([4, 1])

solution, optimal_value = simplex(c, A, b)
print("Optimal solution:", solution)
print("Optimal value:", optimal_value)


Optimal solution: [2. 1.]
Optimal value: -8.0


In [31]:
import numpy as np

def simplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)

    # Create the initial tableau
    tableau = np.zeros((m + 1, n + m + 1))

    # Fill the tableau with coefficients
    tableau[:-1, :-1] = np.hstack((A, np.eye(m)))
    tableau[:-1, -1] = b
    tableau[-1, :n] = -c

    # The main simplex algorithm
    while True:
        # Check if we have an optimal solution (all entries in the objective row are non-negative)
        if np.all(tableau[-1, :-1] >= 0):
            break

        # Pivot column (most negative entry in the objective row)
        pivot_col = np.argmin(tableau[-1, :-1])

        # Pivot row
        ratios = np.divide(tableau[:-1, -1], tableau[:-1, pivot_col], where=(tableau[:-1, pivot_col] > 0))
        valid_ratios = ratios[tableau[:-1, pivot_col] > 0]

        # Check if there's no valid ratio which means unbounded
        if len(valid_ratios) == 0:
            raise ValueError("The problem is unbounded.")

        pivot_row = np.where(ratios == valid_ratios.min())[0][0]

        # Pivot element
        pivot_element = tableau[pivot_row, pivot_col]

        # Update the pivot row
        tableau[pivot_row] /= pivot_element

        # Update the other rows
        for i in range(m + 1):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    # Extract solution
    solution = np.zeros(n)
    for i in range(n):
        col = tableau[:-1, i]
        if np.sum(col == 1) == 1 and np.sum(col == 0) == m - 1:
            solution[i] = tableau[np.where(col == 1)[0][0], -1]

    # Optimal value
    optimal_value = -tableau[-1, -1]

    return solution, optimal_value

# Example Usage
c = np.array([3, 4])
A = np.array([[-3, -1], [-1, -2]])
b = np.array([-4, -4])

try:
    solution, optimal_value = simplex(c, A, b)
    print("Optimal solution:", solution)
    print("Optimal value:", optimal_value)
except ValueError as e:
    print(e)


The problem is unbounded.


In [61]:
def simplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)

    # Create the initial tableau
    tableau = np.zeros((m + 1, n + 1))

    # Fill the tableau with coefficients
    tableau[1:, :-1] = A
    tableau[1:, -1] = b
    tableau[0, :-1] = -c

    while True:
        # Check if we have an optimal solution (all entries in the objective row are negative)

        if np.all(tableau[0, :-1] <= 0):
            break
        # Pivot column (most negative entry in the objective row)
        pivot_col = np.argmax(tableau[0, :-1])



        # Pivot row
        ratios = np.divide(tableau[1:, -1], tableau[1:, pivot_col])

        valid_ratios = ratios[tableau[1:, pivot_col] > 0]

        # Check if there's no valid ratio which means unbounded
        if len(valid_ratios) == 0:
            raise ValueError("The problem is unbounded.")


        ########could be written better################
        pivot_row = np.where(ratios == valid_ratios.min())[0][0] +1

        # Pivot element
        pivot_element = tableau[pivot_row, pivot_col]

        # Update the pivot row
        tableau[pivot_row] /= pivot_element
        # Update the other rows
        for i in range(m + 1):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]
        print(np.round(tableau, 4))
    return tableau


In [62]:
A = np.array([
    [-1, 1, 1, 0, 0],
    [-1, 4, 0, 1, 0],
    [ 3, 1, 0, 0, 1]
])
b = np.array([1, 7,18])
c = np.array([1, -4, 0, 0, 0])
np.round(simplex(c, A, b), 4)

[[ 3.  0. -4.  0.  0. -4.]
 [-1.  1.  1.  0.  0.  1.]
 [ 3.  0. -4.  1.  0.  3.]
 [ 4.  0. -1.  0.  1. 17.]]
[[ 0.      0.      0.     -1.      0.     -7.    ]
 [ 0.      1.     -0.3333  0.3333  0.      2.    ]
 [ 1.      0.     -1.3333  0.3333  0.      1.    ]
 [ 0.      0.      4.3333 -1.3333  1.     13.    ]]


array([[ 0.    ,  0.    ,  0.    , -1.    ,  0.    , -7.    ],
       [ 0.    ,  1.    , -0.3333,  0.3333,  0.    ,  2.    ],
       [ 1.    ,  0.    , -1.3333,  0.3333,  0.    ,  1.    ],
       [ 0.    ,  0.    ,  4.3333, -1.3333,  1.    , 13.    ]])

In [ ]:
A = np.array([[-3, -1, 6], [-1, -2, 5], [2, 4, 5]])
A[A>0]

array([6, 5, 2, 4, 5])

In [ ]:
A[0, :-1]

array([-3, -1])